# Linux Auditd

|         |            |
|---|:-------------|
| Version     | 0.1 |
| Author      | {author}      |
| Contact | {email}      |
| Data tables used | {tablelist here...                                                      }|

## Description:
{the description}


<a id='toc'></a>
## Contents
- [Setup and Authenticate](#setup)

- [First Section](#sectionone)
- [More Sections](#moresections)
- [Appendices](#appendices)
  - [Saving data to Excel](#appendices)


<a id='setup'></a>[Contents](#toc)
# Setup

If you are running this from an Azure Notebooks instance created by ASI you can ignore the *Install Packages* step.


## Install Packages

In [ ]:
# You may needs these - should only need to uncomment and run once
# !pip install msgpack
# !pip install Kqlmagic --no-cache-dir  --upgrade

# !pip install PyHamcrest
# !conda install -c conda-forge python-levenshtein -y
# !conda install requests
# !conda install attrs
# !conda install seaborn
# !conda install bokeh
# !conda install holoviews

# our package
#!pip install ../python --upgrade


### Import Python Packages

In [ ]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)


import numpy as np
from IPython import get_ipython
from IPython.display import display, HTML
import ipywidgets as widgets

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
sns.set()
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
   
import msticpy.sectools as sectools
import msticpy.asitools as asi
import msticpy.asitools.kql as qry
import msticpy.asitools.nbdisplay as asidisp


### Enter or confirm WorkspaceId
To find your Workspace Id go to [Log Analytics](#https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.

In [ ]:
available_workspaces = {'Contoso77':'802d39e1-9d70-404d-832c-2de5e2478eda', 
                        'MSTICLinux':'06dc719f-5dad-47e9-b5af-07d84a0bda4e',
                        'ASIHuntOMSWorkspaceV4': '52b1ab41-869e-4138-9e40-2a4457f09bf0',
                        'ASIHuntOMSWorkspaceV5': '4ca7b24a-6e8f-4540-a8ce-1a80c2948c37',
                        'Rome ILDC - Detection E2E Tests Stage': '3eb61071-5dcd-4db3-94fa-0091a69b7359'}
select_ws = asi.SelectString(description='Select workspace :',
                             item_dict=available_workspaces)

select_ws.display()

In [ ]:
import os
from msticpy.asitools.asiconfig import WorkspaceConfig
ws_config_file = 'config.json'
try:
    ws_config = WorkspaceConfig(ws_config_file)
    print('Found config file')
    for cf_item in ['tenant_id', 'subscription_id', 'resource_group', 'workspace_id', 'workspace_name']:
        print(cf_item, ws_config[cf_item])
except:
    ws_config = None

ws_id = asi.GetEnvironmentKey(env_var='WORKSPACE_ID',
                              prompt='Log Analytics Workspace Id:')
if ws_config:
    ws_id.value = ws_config['workspace_id']
ws_id.display()

### Authenticate to Log Analytics
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. 
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. 
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)
```

In [ ]:
try:
    WORKSPACE_ID = select_ws.value
except NameError:
    try:
        WORKSPACE_ID = ws_id.value
    except NameError:
        WORKSPACE_ID = None
    
if not WORKSPACE_ID:
    raise ValueError('No workspace selected.')

asi.kql.load_kql_magic()

%kql loganalytics://code().workspace(WORKSPACE_ID)

<a id='sectionone'></a>[Contents](#toc)
# Section One

Specify a time range to search for X.

In [ ]:
query_times = asi.QueryTime(units='day', max_before=20, before=5, max_after=1)
query_times.display()

## Linux Events from Auditd

Source Query

In [ ]:
# Linux test
linux_events = r'''
AuditLog_CL
| where Computer startswith 'MSTIC'
| where TimeGenerated > ago(1h)
| extend mssg_parts = extract_all(@"type=(?P<type>[^\s]+)\s+msg=audit\((?P<mssg_id>[^)]+)\):\s+(?P<mssg>[^\r]+)\r?", dynamic(['type', 'mssg_id', 'mssg']), RawData)
| extend mssg_type = tostring(mssg_parts[0][0]), mssg_id = tostring(mssg_parts[0][1])
| project TenantId, TimeGenerated, Computer, mssg_type, mssg_id, mssg_parts
| extend mssg_content = split(mssg_parts[0][2],' ')
| extend typed_mssg = pack(mssg_type, mssg_content)
| summarize AuditdMessage = makelist(typed_mssg) by TenantId, TimeGenerated, Computer, mssg_id'''
%kql -query linux_events
linux_events_df = _kql_raw_result_.to_dataframe()
len(linux_events_df)

We end up with data that looks like this in the AuditMessage column:
```
[{'SYSCALL': ['arch=c000003e',
   'syscall=59',
   'success=yes',
   'exit=0',
   'a0=7f36addfbc18',
   'a1=7f36a934a3f0',
   'a2=7ffd0f830c50',
   'a3=9',
   'items=2',
   'ppid=25281',
   'pid=534',
   'auid=4294967295',
   'uid=0',
   'gid=0',
   'euid=0',
   'suid=0',
   'fsuid=0',
   'egid=0',
   'sgid=0',
   'fsgid=0',
   'tty=(none)',
   'ses=4294967295',
   'comm="sh"',
   'exe="/bin/dash"',
   'key=(null)']},
 {'EXECVE': ['argc=3',
   'a0="/bin/sh"',
   'a1="-c"',
   'a2=69707461626C6573202D2D76657273696F6E']},
 {'CWD': ['cwd="/var/lib/waagent/WALinuxAgent-2.2.35.1"']},
 {'PATH': ['item=0',
   'name="/bin/sh"',
   'inode=26',
   'dev=08:01',
   'mode=0100755',
   'ouid=0',
   'ogid=0',
   'rdev=00:00',
   'nametype=NORMAL',
   'cap_fp=0000000000000000',
   'cap_fi=0000000000000000',
   'cap_fe=0',
   'cap_fver=0']},
 {'PATH': ['item=1',
   'name="/lib64/ld-linux-x86-64.so.2"',
   'inode=2081',
   'dev=08:01',
   'mode=0100755',
   'ouid=0',
   'ogid=0',
   'rdev=00:00',
   'nametype=NORMAL',
   'cap_fp=0000000000000000',
   'cap_fi=0000000000000000',
   'cap_fe=0',
   'cap_fver=0']},
 {'PROCTITLE': ['proctitle=2F62696E2F7368002D630069707461626C6573202D2D76657273696F6E']}]
```

Extraction Logic

In [ ]:
import codecs
from datetime import datetime

encoded_params = {'EXECVE': {'a0', 'a1', 'a2', 'a3', 'arch'},
                  'PROCTITLE': {'proctitle'},
                  'USER_CMD': {'cmd'}}

def unpack_auditd(audit_str):
    event_dict = {}
    for record in audit_str:
        
        for rec_key, rec_val in record.items():
            rec_dict = {}
            encoded_fields_map = encoded_params.get(rec_key, None)
            for rec_item in rec_val:
                rec_split = rec_item.split('=', maxsplit=1)
                if (not encoded_fields_map or rec_split[1].startswith('"') or
                        rec_split[0] not in encoded_fields_map):
                    field_value = rec_split[1].strip('\"')
                else:
                    try:
                        field_value = codecs.decode(rec_split[1], 'hex').decode('utf-8')
                    except:
                        field_value = rec_split[1]
                        print(rec_val)
                        print('ERR:', rec_key, rec_split[0], rec_split[1], type(rec_split[1]))
                rec_dict[rec_split[0]] = field_value
            event_dict[rec_key] = rec_dict
        
    return event_dict

USER_START = {'pid': 'int', 'uid': 'int', 'auid': 'int', 
              'ses': 'int', 'msg': None, 'acct': None, 'exe': None, 
              'hostname': None, 'addr': None, 'terminal': None, 
              'res': None}
FIELD_DEFS = {'SYSCALL': {'success': None, 'ppid': 'int', 'pid': 'int', 
                          'auid': 'int', 'uid': 'int', 'gid': 'int',
                          'euid': 'int', 'egid': 'int', 'ses': 'int',
                          'exe': None, 'com': None},
              'CWD': {'cwd': None},
              'PROCTITLE': {'proctitle': None},
              'LOGIN': {'pid': 'int', 'uid': 'int', 'tty': None, 'old-ses': 'int', 
                        'ses': 'int', 'res': None},
              'EXECVE': {'argc': 'int', 'a0': None, 'a1': None, 'a2': None},
              'USER_START': USER_START,
              'USER_END': USER_START,
              'CRED_DISP': USER_START,
              'USER_ACCT': USER_START,
              'CRED_ACQ': USER_START,
              'USER_CMD': {'pid': 'int', 'uid': 'int', 'auid': 'int', 
                           'ses': 'int', 'msg': None, 'cmd': None,
                           'terminal': None, 'res': None},
             }

def extract_event(message_dict):
    if 'SYSCALL' in message_dict:
        proc_create_dict = {}
        for mssg_type in ['SYSCALL', 'CWD', 'EXECVE', 'PROCTITLE']:
            if (not mssg_type in message_dict or
                    not mssg_type in FIELD_DEFS) :
                continue
            for fieldname, conv in FIELD_DEFS[mssg_type].items():
                value = message_dict[mssg_type].get(fieldname, None)
                if not value:
                    continue
                if conv:
                    if conv == 'int':
                        value = int(value)
                        if value == 4294967295:
                            value = -1
                proc_create_dict[fieldname] = value
            if mssg_type == 'EXECVE':
                args = int(proc_create_dict.get('argc', 1))
                arg_strs = []
                for arg_idx in range(0, args):
                    arg_strs.append(proc_create_dict.get(f'a{arg_idx}', ''))
                    
                proc_create_dict['cmdline'] = ' '.join(arg_strs)
        return 'SYSCALL', proc_create_dict
    else:
        event_dict = {}                                            
        for mssg_type, mssg in message_dict.items():
            if mssg_type in FIELD_DEFS:
                for fieldname, conv in FIELD_DEFS[mssg_type].items():
                    value = message_dict[mssg_type].get(fieldname, None)
                    if conv:
                        if conv == 'int':
                            value = int(value)
                            if value == 4294967295:
                                value = -1
                    event_dict[fieldname] = value
            else:
                event_dict.update(message_dict[mssg_type])
        return list(message_dict.keys())[0], event_dict

def move_cols_to_front(df, column_count):
    """Reorder columms to put the last column count cols to front."""
    return df[list(df.columns[-column_count:]) + list(df.columns[:-column_count])]


def extract_events_to_df(data, event_type=None, verbose=False):
    
    if verbose:
        start_time = datetime.utcnow()
        print(f'Unpacking auditd messages for {len(data)} events...')
    tmp_df = (data.apply(lambda x: extract_event(unpack_auditd(x.AuditdMessage)), 
                         axis=1, result_type='expand')
                  .rename(columns={0: 'EventType', 
                                   1: 'EventData'})
                  )
    # if only one type of event is requested
    if event_type:
        tmp_df = tmp_df[tmp_df['EventType'] == event_type]
        if verbose:
            print(f'Event subset = ', event_type, ' (events: {len(tmp_df)})')
    
    if verbose:
        print('Building output dataframe...')
        
    tmp_df = (tmp_df.apply(lambda x: pd.Series(x.EventData), axis=1)
              .merge(tmp_df[['EventType']], left_index=True, right_index=True)
              .merge(data.drop(['AuditdMessage'], axis=1), 
                 how='inner', left_index=True, right_index=True)
              .dropna(axis=1, how='all'))
    
    if verbose:
        print('Fixing timestamps...')
        
    # extract real timestamp from mssg_id
    tmp_df['TimeStamp'] = (tmp_df.apply(lambda x:
                                        datetime.utcfromtimestamp(float(x.mssg_id.split(':')[0])),
                                        axis=1))
    tmp_df = (tmp_df.drop(['TimeGenerated'], axis=1)
                    .rename(columns={'TimeStamp': 'TimeGenerated'})
                    .pipe(move_cols_to_front, column_count=5))
    if verbose:
        print(f'Complete. {len(tmp_df)} output rows', end=' ')
        delta = datetime.utcnow() - start_time
        print(f'time: {delta.seconds + delta.microseconds/1_000_000} sec')
        
    return tmp_df
                   

def get_event_subset(data, event_type):
    return (data[data['EventType'] == event_type]
             .dropna(axis=1, how='all')
             .infer_objects())

In [ ]:
linux_events_all = extract_events_to_df(linux_events_df, verbose=True)

In [ ]:
sns.set()
(linux_events_all[['EventType', 'TimeGenerated']]
     .groupby('EventType').count().rename(columns={'TimeGenerated': 'EventCount'})
     .sort_values('EventCount', ascending=True)
     .plot.barh(logx=True, figsize=(12,6)))

#### Extract Individual Event Types

In [ ]:
lx_proc_create = get_event_subset(linux_events_all,'SYSCALL')
print(f'{len(proc_create)} Process Create Events')

lx_login = (get_event_subset(linux_events_all, 'LOGIN')
        .merge(get_event_subset(linux_events_all, 'CRED_ACQ'), 
               how='inner',
               left_on=['old-ses', 'pid', 'uid'], 
               right_on=['ses', 'pid', 'uid'],
               suffixes=('', '_cred')).drop(['old-ses','TenantId_cred', 
                                             'Computer_cred'], axis=1)
        .dropna(axis=1, how='all'))
print(f'{len(login)} Login Events')


Other Event Types

In [ ]:
known_type_mask = linux_events_all['EventType'].map(lambda x: x in list(FIELD_DEFS.keys()))
other_events = (linux_events_all[known_type_mask == False])
other_events

### Join Process Creation with Logons

In [ ]:
lx_login.head()

### Merge Process with login data to get account details for processes 

In [ ]:
drop_col_list = ['a0', 'a1', 'a2', 'argc', 'tty',
                 'EventType_log', 'mssg_id_log',
                 'TenantId_log', 'Computer_log',
                 'TimeGenerated_cred', 'res_cred', 'ses_cred', 'msg']
proc_create_log = lx_proc_create.merge(lx_login, 
                                    how='left',
                                    left_on=['ses'], right_on=['ses'],
                                    suffixes=('', '_log')).drop(drop_col_list, axis=1)

len(proc_create_log)
proc_create_log[pd.notna(proc_create_log['uid_log'])]

<a id='linuxLogins'></a>[Contents](#toc)
# Linux Logins with IP Address

In [ ]:
lx_login[lx_login['addr'] != '?'][['Computer', 'TimeGenerated', 
                                   'pid', 'ses', 'acct', 'addr', 
                                   'exe', 'hostname', 'msg',
                                   'res_cred', 'ses_cred', 'terminal']]

### Field name mapping

In [ ]:
proc_create_log

In [ ]:
lx_to_proc_create = {'acct': 'SubjectUserName',
                     'uid': 'SubjectUserSid',
                     'user': 'SubjectUserName',
                     'ses': 'SubjectLogonId',
                     'pid': 'NewProcessId',
                     'exe': 'NewProcessName',
                     'ppid': 'ProcessId',
                     'cmdline': 'CommandLine',}

proc_create_to_lx = {'SubjectUserName': 'acct',
                     'SubjectUserSid': 'uid',
                     'SubjectUserName': 'user',
                     'SubjectLogonId': 'ses',
                     'NewProcessId': 'pid',
                     'NewProcessName': 'exe',
                     'ProcessId': 'ppid',
                     'CommandLine': 'cmdline',}

lx_to_logon = {'acct': 'SubjectUserName',
               'auid': 'SubjectUserSid',
               'user': 'TargetUserName',
               'uid': 'TargetUserSid',
               'ses': 'TargetLogonId',
               'exe': 'LogonProcessName',
               'terminal': 'LogonType',
               'msg': 'AuthenticationPackageName',
               'res': 'Status',
               'addr': 'IpAddress',
               'hostname': 'WorkstationName',}

logon_to_lx = {'SubjectUserName': 'acct',
               'SubjectUserSid': 'auid',
               'TargetUserName': 'user',
               'TargetUserSid': 'uid',
               'TargetLogonId': 'ses',
               'LogonProcessName': 'exe',
               'LogonType': 'terminal',
               'AuthenticationPackageName': 'msg',
               'Status': 'res',
               'IpAddress': 'addr',
               'WorkstationName': 'hostname',}

lx_proc_create_trans = lx_proc_create.rename(columns=lx_to_proc_create)
lx_login_trans = lx_login.rename(columns=lx_to_logon)

## Find Distinct Processes in Process Data

In [ ]:
from msticpy.sectools.eventcluster import dbcluster_events, add_process_features

feature_procs_h1 = add_process_features(input_frame=lx_proc_create_trans,
                                        path_separator='/')


# you might need to play around with the max_cluster_distance parameter.
# decreasing this gives more clusters.
(clus_events, dbcluster, x_data) = dbcluster_events(data=feature_procs_h1,
                                                    cluster_columns=['commandlineTokensFull', 
                                                                     'pathScore', 
                                                                     'isSystemSession',
                                                                     'SubjectLogonId'],
                                                    time_column='TimeGenerated',
                                                    max_cluster_distance=0.0001)
print('Number of input events:', len(feature_procs_h1))
print('Number of clustered events:', len(clus_events))
(clus_events.sort_values('TimeGenerated')[['TimeGenerated', 'LastEventTime',
                                           'NewProcessName', 'CommandLine', 
                                           'ClusterSize', 'commandlineTokensFull',
                                           'SubjectLogonId',
                                           'pathScore', 'isSystemSession']]
    .sort_values('ClusterSize', ascending=True))

<a id='appendices'></a>[Contents](#toc)
# Appendices

## Available DataFrames

In [ ]:
print('List of current DataFrames in Notebook')
print('-' * 50)
current_vars = list(locals().keys())
for var_name in current_vars:
    if isinstance(locals()[var_name], pd.DataFrame) and not var_name.startswith('_'):
        print(var_name)

## Saving Data to CSV
To save the contents of a pandas DataFrame to an CSV
use the following syntax
```
host_logons.to_csv('host_logons.csv')
```

## Saving Data to Excel
To save the contents of a pandas DataFrame to an Excel spreadsheet
use the following syntax
```
writer = pd.ExcelWriter('myWorksheet.xlsx')
my_data_frame.to_excel(writer,'Sheet1')
writer.save()
```